In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq


In [2]:
country_list = pd.read_csv('./data/CensusCountryMapping_FRBAtl_short.csv', dtype={'CTY_CODE': str})

country_list.loc[len(country_list)] = ["0003", "EUE", "EUROPEAN UNION"]

country_list.loc[len(country_list)] = ["5082", "GZA", "Gaza"]

country_list.loc[len(country_list)] = ["5083", "WBK", "West Bank"]

country_list = country_list.dropna()

country_list["tariff"] = 10.0

country_list.loc[country_list["CTY_CODE"] == "5700", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "5820", "tariff"] = 30.0

country_list.loc[country_list["CTY_CODE"] == "1220", "tariff"] = 8.75

country_list.loc[country_list["CTY_CODE"] == "2010", "tariff"] = 4.5

country_list.loc[country_list["CTY_CODE"] == "0003", "tariff"] = 10.0

recipricol = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv', dtype={'CTY_CODE': str})

recipricol.rename(columns={"tariff_rate": "rtariff"}, inplace = True)

country_list = pd.merge(left = country_list, right = recipricol[["rtariff", "CTY_CODE"]], on = "CTY_CODE", how = "left")

country_list["rtariff"] = country_list["rtariff"].fillna(10.0)

country_list.loc[country_list["CTY_CODE"] == "5700", "rtariff"] = 34.0

country_list.loc[country_list["CTY_CODE"] == "5820", "rtariff"] = 34.0
#IF EU you get 50 and you'll like it

country_list["country"] = country_list["country"].str.upper()


In [3]:
country_list.head(25)

,CTY_CODE,ISO3,country,tariff,rtariff
0,4271,AND,ANDORRA,10.0,10.0
1,5200,ARE,UNITED ARAB EMIRATES,10.0,10.0
2,5310,AFG,AFGHANISTAN,10.0,10.0
3,2484,ATG,ANTIGUA AND BARBUDA,10.0,10.0
4,2481,AIA,ANGUILLA,10.0,10.0
5,4810,ALB,ALBANIA,10.0,10.0
6,4631,ARM,ARMENIA,10.0,10.0
7,7620,AGO,ANGOLA,10.0,32.0
8,3570,ARG,ARGENTINA,10.0,10.0
9,4330,AUT,AUSTRIA,10.0,10.0


In [4]:
new_list = pd.read_csv('./tariff-lists/clean-august-7-tariffs.csv')

new_list.rename({"country_name":"country"}, axis = 1, inplace = True)

new_list["country"] = new_list["country"].str.upper()

country_list = pd.merge(country_list, new_list, on="country", how="left")

country_list["rtariff-aug"] = country_list["rtariff-aug"].fillna(country_list["rtariff"])

country_list.loc[country_list["country"] == "EUROPEAN UNION", "rtariff-aug"] = 15.0

country_list.loc[country_list["country"] == "KOREA, SOUTH", "rtariff-aug"] = 15.0

country_list.loc[country_list["country"] == "INDIA", "rtariff-aug"] = 50.0


In [5]:
eu_iso3_codes = [
    "AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN",
    "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX",
    "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"
]

country_list.loc[country_list["ISO3"].isin(eu_iso3_codes), "rtariff-aug"] = 15.0

In [6]:
country_list.loc[country_list["country"] == "EUROPEAN UNION"]

,CTY_CODE,ISO3,country,tariff,rtariff,rtariff-aug
223,0003,EUE,EUROPEAN UNION,10.0,20.0,15.0


In [7]:
expemption_list = pd.read_csv('./tariff-lists/annex-II-exemptions.csv', dtype={'HTSUS': str})
# these are the annex II exemptions from the reciprical tariff list

expemption_list_phones = pd.read_csv('./tariff-lists/41225_exemptions.csv', dtype={'HTSUS': str})
# these are the phone exemptions

steel_list = pd.read_csv('./tariff-lists/steel-tariffs.csv', dtype={'HTSUS': str})
# these are the steel products subject to the new 232 tariffs. This is at 8 digit level.

alu_list = pd.read_csv('./tariff-lists/alu-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

alu_8_list = alu_list[alu_list['HTSUS'].str.len() == 8]

auto_list = pd.read_csv('./tariff-lists/auto-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

auto_6_list = auto_list[auto_list['HTSUS'].str.len() == 6]

auto_8_list = auto_list[auto_list['HTSUS'].str.len() == 8]

copper_list = pd.read_csv('./tariff-lists/copper-list.csv', dtype={'HTSUS': str})

# country_list = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv',dtype={'CTY_CODE': str})

In [8]:
def weighted_average(df, value_col, weight_col):
    """
    Compute the weighted average of `value_col` using `weight_col` as weights.
    """
    return (df[value_col] * df[weight_col]).sum() / df[weight_col].sum()

In [9]:
def get_all_imports(cnty_code, tariff, ISO3):

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    hs = 'HS10'

    end_use = "hs?get=CTY_NAME,CON_VAL_YR,CAL_DUT_YR,I_COMMODITY,I_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

    surl  = surl + my_key + "&time=" + "2024-12" + "&COMM_LVL=" + hs 

    url = surl + "&CTY_CODE=" + cnty_code

    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    #######################################################################

    df.time = pd.to_datetime(df.time, format="%Y-%m")

    df["imports"] = df["CON_VAL_YR"].astype(float)

    df["duty"] = df["CAL_DUT_YR"].astype(float)

    df["share"] = df["imports"] / df["imports"].sum()
            
    df[hs] = df["I_COMMODITY"].astype(str)

    df["HS8"] = df["HS10"].str[0:8]

    df["HS6"] = df["HS10"].str[0:6]

    df["HS2"] = df["HS10"].str[0:2]

    #######################################################################
    # this then filters so products are only those that are subject to the new tariffs
    # the reciprocal exemptions, steel, aluminum and auto tariffs.

    df["tariff"] = tariff

    exemption = df["HS10"].isin(expemption_list["HTSUS"].tolist()) 

    df.loc[exemption, "tariff"]  = 0.0

    ####################################################
    # this is the phone exemption list. This is a 10 digit list.

    exemption_phones = df["HS10"].isin(expemption_list_phones["HTSUS"].tolist()) 

    if cnty_code == "5700": # if china then still Fentanyl 20 percent on the phones

        df.loc[exemption_phones, "tariff"] = 20.0

    else:

        df.loc[exemption_phones, "tariff"] = 0.0

    ####################################################

    steel = df["HS8"].isin(steel_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase

        df.loc[steel, "tariff"]  = 25.0 

    else:

        df.loc[steel, "tariff"]  = 50.0 


    ####################################################

    alu = df["HS10"].isin(alu_list["HTSUS"].tolist()) | df["HS8"].isin(alu_8_list["HTSUS"].tolist())

    if cnty_code == "4120": # if the UK, then exempted from the increase
    
        df.loc[alu, "tariff"]  = 25.0 

    else:

        df.loc[alu, "tariff"]  = 50.0
        

    ####################################################


    auto = df["HS8"].isin(auto_8_list["HTSUS"].tolist()) | df["HS10"].isin(auto_list["HTSUS"].tolist()) | df["HS6"].isin(auto_6_list["HTSUS"].tolist())

    if cnty_code == "1220": # if canada or mexico, then assumption is no tariff

        df.loc[auto, "tariff"]  = 0.0

    elif cnty_code == "2010":

        df.loc[auto, "tariff"]  = 0.0

    elif cnty_code == "5880":
    # Japan gets an auto deal

        df.loc[auto, "tariff"]  = 15.0

    elif cnty_code == "0003":
    #EU gets and auto deal

        df.loc[auto, "tariff"]  = 15.0

    elif ISO3 in eu_iso3_codes:

        df.loc[auto, "tariff"]  = 15.0

    elif cnty_code == "5800":
        #Korea gets an auto deal

        df.loc[auto, "tariff"]  = 15.0

    else:
        df.loc[auto, "tariff"]  = 25.0


    ####################################################
            
    copper = df["HS8"].isin(copper_list["HTSUS"].tolist())

    df.loc[copper, "tariff"]  = 50.0

    ####################################################


    return df

In [ ]:
for index, row in country_list.iterrows():

    print(f"Processing {row['country']}")

    out_file = f'./data-hs2/hs2-tariff-{row["CTY_CODE"]}.csv'

    # if os.path.exists(out_file):
        
    #     print("Already have downloaded file")
        
    #     continue
    
    df = get_all_imports(row["CTY_CODE"], row["rtariff-aug"], row["ISO3"])

    dfhs2 = df.groupby("HS2").agg({"imports": "sum", "duty": "sum", "CTY_NAME": "first", 
                              "CTY_CODE": "first"}).reset_index()

    foo = df.groupby("HS2").apply(lambda x: weighted_average(x, "tariff", "imports"))

    dfhs2 = dfhs2.merge(foo.rename("2025_tariff_increase"), left_on="HS2", right_index=True)

    dfhs2.rename(columns={"imports": "2024_imports", "duty": "2024_duty"}, inplace=True)

    dfhs2["2024_tariff"] = 100* ( dfhs2["2024_duty"] / dfhs2["2024_imports"] )

    dfhs2.to_csv(out_file, index=False)

Processing ANDORRA
<Response [200]>
Processing UNITED ARAB EMIRATES
<Response [200]>
Processing AFGHANISTAN
<Response [200]>
Processing ANTIGUA AND BARBUDA
<Response [200]>
Processing ANGUILLA
<Response [200]>
Processing ALBANIA
<Response [200]>
Processing ARMENIA
<Response [200]>
Processing ANGOLA
<Response [200]>
Processing ARGENTINA
<Response [200]>
Processing AUSTRIA
<Response [200]>
Processing AUSTRALIA
<Response [200]>
Processing ARUBA
<Response [200]>
Processing AZERBAIJAN
<Response [200]>
Processing BOSNIA AND HERZEGOVINA
<Response [200]>
Processing BARBADOS
<Response [200]>
Processing BANGLADESH
<Response [200]>
Processing BELGIUM
<Response [200]>
Processing BURKINA FASO
<Response [200]>
Processing BULGARIA
<Response [200]>
Processing BAHRAIN
<Response [200]>
Processing BURUNDI
<Response [200]>
Processing BENIN
<Response [200]>
Processing BERMUDA
<Response [200]>
Processing BRUNEI DARUSSALAM
<Response [200]>
Processing BOLIVIA
<Response [200]>
Processing BRAZIL
<Response [200]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


<Response [200]>
Processing DOMINICA
<Response [200]>
Processing DOMINICAN REPUBLIC
<Response [200]>
Processing ALGERIA
<Response [200]>
Processing ECUADOR
<Response [200]>
Processing ESTONIA
<Response [200]>
Processing EGYPT
<Response [200]>
Processing ERITREA
<Response [200]>
Processing SPAIN
<Response [200]>
Processing ETHIOPIA
<Response [200]>
Processing FINLAND
<Response [200]>
Processing FIJI
<Response [200]>
Processing FALKLAND ISLANDS
<Response [200]>
Processing MICRONESIA, FED. STS.
<Response [200]>
Processing FAROE ISLANDS
<Response [200]>
Processing FRANCE
<Response [200]>
Processing GABON
<Response [200]>
Processing UNITED KINGDOM
<Response [200]>
Processing GRENADA
<Response [200]>
Processing GEORGIA
<Response [200]>
Processing FRENCH GUIANA
<Response [200]>
Processing GHANA
<Response [200]>
Processing GIBRALTAR
<Response [200]>
Processing GREENLAND
<Response [200]>
Processing GAMBIA
<Response [200]>
Processing GUINEA
<Response [200]>
Processing GUADELOUPE
<Response [200]>

In [ ]:
checkdf = pd.read_csv(f'./data-hs2/hs2-tariff-1220.csv')

In [ ]:
(checkdf["2025_tariff_increase"] * checkdf["2024_imports"]).sum() / checkdf["2024_imports"].sum()

6.408504722644438

In [ ]:
checkdf["2024_duty"].sum() / checkdf["2024_imports"].sum()

0.001004719282005802

In [ ]:
checkdf[checkdf["HS2"] == 87]

,HS2,2024_imports,2024_duty,CTY_NAME,CTY_CODE,2025_tariff_increase,2024_tariff
85,87,5.038833e+10,76997362.0,CANADA,1220,1.478907,0.152808
